In [40]:
from agents import SQLiteSession

session = SQLiteSession("user_123", "src/openai_study_mode_clone/study_agent_session.db")

msg = await session.get_items()

In [41]:
print(msg)

[{'content': 'i want to lean python', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': "Awesome! Python is a great choice.\n\nTo get started, what's your experience with programming? Have you coded before, or is this your first time?", 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': 'hy', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': "Got it. Since you're new to coding, we'll start with the basics. Here’s the plan:\n\n1.  **What is Python?** Briefly, what it is and why it's popular.\n2.  **Basic Syntax:** Learn to write simple Python commands.\n3.  **Variables:** How to store and use data.\n4.  **Mini-exercise:** A small coding problem to check your understanding.\n\nReady? First, what do you think Python is used for? Any ideas?", 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': 'ok', 'role': 'user'}, {'id': '__fake_id__

In [ ]:
for item in msg:
    if item["role"] == "user":
        print(f"User message found: {item['content']}")
    # messages = item['messages']
    if item["role"] == "assistant":
        # print(f"Assistant message found: {item.get('content')}")
        message = item.get('content')[0].get('text')
        print(f"Assistant message found: {message}")

In [5]:
from agents import AsyncOpenAI, OpenAIChatCompletionsModel
from dotenv import load_dotenv
import os

# Load environment variables from a .env file
load_dotenv()

def get_llm_client() -> AsyncOpenAI:
    """
    Initializes and returns the asynchronous LLM client.

    Raises:
        ValueError: If the required API key or base URL environment variables are not set.

    Returns:
        AsyncOpenAI: An instance of the async LLM client.
    """
    # The API key for the service. The variable name suggests a Gemini model provider
    # that uses an OpenAI-compatible API.
    api_key = os.environ.get("GEMINI_API_KEY")
    if not api_key:
        raise ValueError("GEMINI_API_KEY environment variable not set.")

    # The base URL for the chat completion endpoint.
    base_url = os.environ.get("LLM_CHAT_COMPLETION_URL")
    if not base_url:
        raise ValueError("LLM_CHAT_COMPLETION_URL environment variable not set.")

    return AsyncOpenAI(api_key=api_key, base_url=base_url)

def get_llm_model(client: AsyncOpenAI) -> OpenAIChatCompletionsModel:
    """
    Initializes and returns the chat completions model.

    Args:
        client (AsyncOpenAI): The client to use for the model.

    Returns:
        OpenAIChatCompletionsModel: An instance of the chat completions model.
    """
    model_name = os.environ.get("LLM_MODEL", "gemini-2.0-flash")
    return OpenAIChatCompletionsModel(openai_client=client, model=model_name)

# Create singleton instances to be imported by other modules
client: AsyncOpenAI = get_llm_client()
model: OpenAIChatCompletionsModel = get_llm_model(client)


In [ ]:
from agents import Agent, Runner , SQLiteSession , ItemHelpers , 
# from openai_study_mode_clone.config.llm_model_config import model

study_agent : Agent = Agent(
    name="Study Agent",
    instructions="""
    You are a helpful assistant for a student studying a specific topic. Your goal is to provide explanations, answer questions, and guide the student through their learning process.
---""",
    model=model,
)


async def study_agent_service(user_input: str , user_id: str) -> str:
    # session = SQLiteSession(user_id, "study_agent_session.db")
    response = await Runner(
        starting_agent=study_agent,
        input=user_input,
        # session=session
    )
    # session.close()
    # return response.final_output


In [17]:
from openai.types.responses import ResponseTextDeltaEvent

responses = Runner.run_streamed(
    starting_agent=study_agent,
    input="What is the Pythagorean theorem?",
    # session=session
)
print("=== Streaming Responses ===")
print("=== Run started ===")
print("=== Processing events ===")
async for event in responses.stream_events():
    if event.type == "raw_response_event" and isinstance( event.data,ResponseTextDeltaEvent ):
        print(f"Text chunk: {event.data.delta}")
    # print(f"Event: {event}")
    # # print(f"Event data: {event.data}")
    # print(f"Event type: {event.type}")
    # print("=== Event processed ===")

=== Streaming Responses ===
=== Run started ===
=== Processing events ===
Text chunk: Okay
Text chunk: , let's talk about the Pythagorean Theorem!

**In a nutshell:**
Text chunk:  The Pythagorean Theorem is a fundamental relationship in geometry that connects the sides of a right
Text chunk:  triangle.

**What it says:** In a right triangle (a triangle with one angle that is exactly 90 degrees), the square of the length of
Text chunk:  the longest side (called the hypotenuse) is equal to the sum of the squares of the lengths of the other two sides (called the legs or cathet
Text chunk: us).

**The Formula:**
a² + b² = c²

Where:
*   `a` and `b` are the lengths of the two shorter sides (legs) of the right triangle.
*   `c`
Text chunk:  is the length of the longest side (hypotenuse), which is opposite the right angle.

**Let's break it down with a simple example:**

Imagine a right triangle where one leg (`a`) is 3 units long and
Text chunk:  the other leg (`b`) is 4 units long.

1.  **S

In [ ]:
async def study_agent_service(user_input: str , user_id: str):
     responses = Runner.run_streamed(
        starting_agent=study_agent,
        input=user_input,
        # session=session
    )
    
    for event in responses.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
            # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            (f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types

            print("=== Run complete ===")

In [22]:
import os
from dotenv import load_dotenv, find_dotenv
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, function_tool, AgentHooks, RunContextWrapper

_: bool = load_dotenv(find_dotenv())

# ONLY FOR TRACING
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "")

gemini_api_key: str = os.getenv("GEMINI_API_KEY", "")

# 1. Which LLM Service?
external_client: AsyncOpenAI = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# 2. Which LLM Model?
llm_model: OpenAIChatCompletionsModel = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash",
    openai_client=external_client
)


# Agent Lifecycle Callbacks/Hooks
class HelloAgentHooks(AgentHooks):
    def __init__(self, lifecycle_name: str):
        self.lifecycle_name = lifecycle_name
        
    async def on_start(self, context, agent):
        context.context
        print(f"\n\n[{self.lifecycle_name}] Agent {agent.name} starting with context: {context}\n\n")
        
    async def on_llm_start(self, context: RunContextWrapper, agent, system_prompt, input_items):
        print(f"\n\n[{self.lifecycle_name}] LLM call starting with system prompt: {system_prompt} and input items: {input_items}\n\n")
        
    async def on_llm_end(self, context, agent, response):
        print(f"\n\n[{self.lifecycle_name}] LLM call ended with response: {response}\n\n")
        
    async def on_end(self, context, agent, output):
        print(f"\n\n[{self.lifecycle_name}] Agent {agent.name} ended with output: {output}\n\n")


@function_tool
def get_weather(city: str) -> str:
    """A simple function to get the weather for a user."""
    return f"The weather for {city} is sunny."



news_agent: Agent = Agent(
    name="NewsAgent",
    instructions="You are a helpful news assistant.",
    model=llm_model,
    hooks=HelloAgentHooks("NewsAgentLifecycle")
)


base_agent: Agent = Agent(
    name="WeatherAgent",
    instructions="You are a helpful assistant. Talk about weather and let news_agent handle the news things",
    model=llm_model,
    tools=[get_weather],
    hooks=HelloAgentHooks("WeatherAgentLifecycle"),
    handoffs=[news_agent]
)

res = Runner.run_streamed(base_agent, "What's the latest news about Qwen Code - seems like it can give though time to claude code.")
# print(res.last_agent.name)
# print(res.final_output)

# Now check the trace in 

async for event in responses.stream_events():
    # We'll ignore the raw responses event deltas
    if event.type == "raw_response_event":
        continue
    # When the agent updates, print that
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
        continue
    # When items are generated, print them
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print("-- Tool was called")
        elif event.item.type == "tool_call_output_item":
            print(f"-- Tool output: {event.item.output}")
        elif event.item.type == "message_output_item":
            print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
        else:
            pass  # Ignore other event types

            print("=== Run complete ===")



[WeatherAgentLifecycle] Agent WeatherAgent starting with context: RunContextWrapper(context=None, usage=Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0))




[WeatherAgentLifecycle] LLM call starting with system prompt: You are a helpful assistant. Talk about weather and let news_agent handle the news things and input items: [{'content': "What's the latest news about Qwen Code - seems like it can give though time to claude code.", 'role': 'user'}]




[WeatherAgentLifecycle] LLM call ended with response: ModelResponse(output=[ResponseFunctionToolCall(arguments='{}', call_id='function-call-757011918939407599', name='transfer_to_newsagent', type='function_call', id='__fake_id__', status=None)], usage=Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(